In [5]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import optuna
import plotly.express as px
import random

# Load and prepare data
DATA_PATH = 'german_credit_data_cleaned.csv'
df = pd.read_csv(DATA_PATH)
y = df['Risk_good'].values
X_full = df.drop(columns=['Risk_good'])
sensitive = df['Sex_male'].values



In [6]:
GLOBAL_SEED = 873
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

In [7]:
# Define the custom objective function factory
def make_custom_obj(sens_array, alpha=0.5):
    def custom_obj(preds, dmatrix):
        labels = dmatrix.get_label()
        preds_prob = 1.0 / (1.0 + np.exp(-preds))
        grad_log = preds_prob - labels
        hess_log = preds_prob * (1.0 - preds_prob)
        mask_p = (sens_array == 1)
        mask_u = (sens_array == 0)
        n_p, n_u = mask_p.sum(), mask_u.sum()
        mean_p = preds_prob[mask_p].mean() if n_p > 0 else 0
        mean_u = preds_prob[mask_u].mean() if n_u > 0 else 0
        diff = mean_p - mean_u
        sign = np.sign(diff)
        sigmoid_deriv = preds_prob * (1.0 - preds_prob)
        grad_fair = np.zeros_like(preds)
        grad_fair[mask_p] = sign * (1.0 / n_p) * sigmoid_deriv[mask_p] if n_p > 0 else 0
        grad_fair[mask_u] = -sign * (1.0 / n_u) * sigmoid_deriv[mask_u] if n_u > 0 else 0
        hess_fair = np.zeros_like(preds)
        grad = alpha * grad_log + (1 - alpha) * grad_fair
        hess = alpha * hess_log + (1 - alpha) * hess_fair
        return grad, hess
    return custom_obj

# Define the objective function for Optuna

def create_objective_function(alpha,seed=GLOBAL_SEED):
    def objective(trial):
        # Suggest hyperparameters
        # alpha = trial.suggest_float('alpha', 0.0, 1.0)
        max_depth = trial.suggest_int('max_depth', 3, 7)
        eta = trial.suggest_float('eta', 0.01, 0.3)
        subsample = trial.suggest_float('subsample', 0.6, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)

        # Initialize cross-validation
        kf = KFold(n_splits=3, shuffle=True, random_state=seed)
        acc_scores = []
        dpd_scores = []

        # Perform 3-fold cross-validation
        for train_idx, val_idx in kf.split(X_full):
            # Split data
            X_train = X_full.iloc[train_idx].drop(columns=['Sex_male']).values
            y_train = y[train_idx]
            sens_train = sensitive[train_idx]
            X_val = X_full.iloc[val_idx].drop(columns=['Sex_male']).values
            y_val = y[val_idx]
            sens_val = sensitive[val_idx]

            # Create custom objective
            train_obj = make_custom_obj(sens_train, alpha=alpha)

            # Set up DMatrix
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)

            # Train the model
            params = {
                'max_depth': max_depth,
                'eta': eta,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'verbosity': 0,
                'seed': seed + trial.number
            }
            bst = xgb.train(
                params,
                dtrain,
                num_boost_round=100,
                obj=train_obj,
                evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=10,
                verbose_eval=False
            )

            # Predict on validation set
            pred_prob = bst.predict(dval)
            pred = (pred_prob > 0.5).astype(int)

            # Compute accuracy and DPD
            acc = accuracy_score(y_val, pred)
            dpd_val = abs(pred[sens_val == 1].mean() - pred[sens_val == 0].mean())
            acc_scores.append(acc)
            dpd_scores.append(dpd_val)

        # Compute mean metrics across folds
        mean_acc = np.mean(acc_scores)
        mean_dpd = np.mean(dpd_scores)

        # Store metrics for analysis
        trial.set_user_attr('accuracy', mean_acc)
        trial.set_user_attr('dpd', mean_dpd)
        trial.set_user_attr('params', {
            'alpha': alpha,
            'max_depth': max_depth,
            'eta': eta,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree
        })

        return mean_acc, mean_dpd
    return objective


In [ ]:
# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

def normalise(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)
#adaptive weights based on distance

results = []
count_alpha = 5
trials_per_alpha = 20
sampler = optuna.samplers.NSGAIISampler(seed=GLOBAL_SEED)
study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
objective = create_objective_function(alpha=0, seed=GLOBAL_SEED)  
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=0")
max_val = 0
for trial in study.best_trials:
    val = test_loss(0,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 0,
    'params': best_trial.user_attrs['params']
    })
study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
objective = create_objective_function(alpha=1, seed=GLOBAL_SEED)  
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=1")
max_val = 0
for trial in study.best_trials:
    val = test_loss(1,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 1,
    'params': best_trial.user_attrs['params']
})
print(results)

max_acc = max([res['accuracy'] for res in results])
max_dpd = max([res['dpd'] for res in results])
min_acc = min([res['accuracy'] for res in results])
min_dpd = min([res['dpd'] for res in results])




for j in range(count_alpha):
    print(j)
    
    results.sort(key=lambda x: x['alpha'])
    max_dist = 0
    print(f"max_acc: {max_acc}, max_dpd: {max_dpd}, min_acc: {min_acc}, min_dpd: {min_dpd}")
    for i in range(len(results)-1):
        acc_dist = (normalise(results[i]['accuracy'],min_acc,max_acc)-normalise(results[i+1]['accuracy'],min_acc,max_acc))**2
        dpd_dist = (normalise(results[i]['dpd'],min_dpd,max_dpd)-normalise(results[i+1]['dpd'],min_dpd,max_dpd))**2

        dist = np.sqrt(acc_dist + dpd_dist)
        if dist > max_dist:
            max_dist = dist
            print(f"max dist: {max_dist}, acc_dist: {acc_dist}, dpd_dist: {dpd_dist}")
            alpha = (results[i]['alpha']+results[i+1]['alpha'])/2
    study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
    objective = create_objective_function(alpha=alpha, seed=GLOBAL_SEED)  
    study.optimize(objective, n_trials=trials_per_alpha, )
    # print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
        
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })
    max_acc = max(max_acc, best_trial.values[0])
    max_dpd = max(max_dpd, best_trial.values[1])
    min_acc = min(min_acc, best_trial.values[0])
    min_dpd = min(min_dpd, best_trial.values[1])
acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-16 19:27:43,335] A new study created in memory with name: no-name-3fac77d4-27b7-4814-b1c5-83363e66b715
[I 2025-05-16 19:27:43,514] Trial 0 finished with values: [0.5000091902473095, 0.0] and parameters: {'max_depth': 4, 'eta': 0.23427942936231477, 'subsample': 0.6945965931961444, 'colsample_bytree': 0.6527247741457783}.
[I 2025-05-16 19:27:43,519] A new study created in memory with name: no-name-7c1152c8-a5c9-4200-9106-447505ca8b25
[I 2025-05-16 19:27:43,649] Trial 0 finished with values: [0.661435884239645, 0.07780119383333745] and parameters: {'max_depth': 3, 'eta': 0.14502880540829954, 'subsample': 0.6831438346551075, 'colsample_bytree': 0.6650056059966459}.
[I 2025-05-16 19:27:43,653] A new study created in memory with name: no-name-e98662f0-8f6b-4a70-8d68-7bb27eff506b


Completed study for alpha=0
Completed study for alpha=1
[{'accuracy': 0.5000091902473095, 'dpd': 0.0, 'alpha': 0, 'params': {'alpha': 0, 'max_depth': 4, 'eta': 0.23427942936231477, 'subsample': 0.6945965931961444, 'colsample_bytree': 0.6527247741457783}}, {'accuracy': 0.661435884239645, 'dpd': 0.07780119383333745, 'alpha': 1, 'params': {'alpha': 1, 'max_depth': 3, 'eta': 0.14502880540829954, 'subsample': 0.6831438346551075, 'colsample_bytree': 0.6650056059966459}}]
0
max_acc: 0.661435884239645, max_dpd: 0.07780119383333745, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 1.4142135623730951, acc_dist: 1.0, dpd_dist: 1.0


[I 2025-05-16 19:27:43,773] Trial 0 finished with values: [0.6642986462765713, 0.037019707718247245] and parameters: {'max_depth': 7, 'eta': 0.2189983990110477, 'subsample': 0.7281971459251811, 'colsample_bytree': 0.958988210314949}.
[I 2025-05-16 19:27:43,776] A new study created in memory with name: no-name-c10ea187-3897-4672-a61c-b4caad054a48
[I 2025-05-16 19:27:43,913] Trial 0 finished with values: [0.6628664994041656, 0.028188257559743534] and parameters: {'max_depth': 3, 'eta': 0.0380102539744466, 'subsample': 0.7932399343106844, 'colsample_bytree': 0.7689704204658687}.
[I 2025-05-16 19:27:43,916] A new study created in memory with name: no-name-9aadab0a-a4c1-4d8e-9b2d-d65c40afb937


1
max_acc: 0.6642986462765713, max_dpd: 0.07780119383333745, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 1.1074334634935452, acc_dist: 1.0, dpd_dist: 0.2264088760653095
2
max_acc: 0.6642986462765713, max_dpd: 0.07780119383333745, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 1.1074334634935452, acc_dist: 1.0, dpd_dist: 0.2264088760653095


[I 2025-05-16 19:27:44,047] Trial 0 finished with values: [0.6635802752785411, 0.027068539729331025] and parameters: {'max_depth': 3, 'eta': 0.05323929968683921, 'subsample': 0.7140418773899931, 'colsample_bytree': 0.7577135900044464}.
[I 2025-05-16 19:27:44,050] A new study created in memory with name: no-name-fa93c761-8ff1-4461-bec5-4dad95283f4e
[I 2025-05-16 19:27:44,213] Trial 0 finished with values: [0.6678659939405636, 0.04443812101285003] and parameters: {'max_depth': 7, 'eta': 0.27371053232305487, 'subsample': 0.7114829952902549, 'colsample_bytree': 0.7009075119363926}.
[I 2025-05-16 19:27:44,215] A new study created in memory with name: no-name-51be83b9-3c39-45b2-9c64-1789183ed905


...

496
max_acc: 0.6700134483952297, max_dpd: 0.07780119383333745, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 1.0771757541325098, acc_dist: 0.9338987292256314, dpd_dist: 0.2264088760653095
497
max_acc: 0.6700134483952297, max_dpd: 0.07780119383333745, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 1.0771757541325098, acc_dist: 0.9338987292256314, dpd_dist: 0.2264088760653095


[I 2025-05-16 19:29:18,275] Trial 0 finished with values: [0.6657216029016674, 0.04186240511471331] and parameters: {'max_depth': 7, 'eta': 0.09228703495434147, 'subsample': 0.6089034909087588, 'colsample_bytree': 0.6005007599358635}.
[I 2025-05-16 19:29:18,282] A new study created in memory with name: no-name-b7d9c2fa-258b-44ab-96a3-1c5576c9347a
[I 2025-05-16 19:29:18,417] Trial 0 finished with values: [0.6642955828608015, 0.047172657266691585] and parameters: {'max_depth': 5, 'eta': 0.28557761492382033, 'subsample': 0.9428898533991945, 'colsample_bytree': 0.7190489276592553}.
[I 2025-05-16 19:29:18,423] A new study created in memory with name: no-name-14f10800-8055-4178-acfb-73409d678a00


498
max_acc: 0.6700134483952297, max_dpd: 0.07780119383333745, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 1.0771757541325098, acc_dist: 0.9338987292256314, dpd_dist: 0.2264088760653095
499
max_acc: 0.6700134483952297, max_dpd: 0.07780119383333745, min_acc: 0.5000091902473095, min_dpd: 0.0
max dist: 1.0771757541325098, acc_dist: 0.9338987292256314, dpd_dist: 0.2264088760653095


[I 2025-05-16 19:29:18,541] Trial 0 finished with values: [0.663581806986426, 0.036893291584953164] and parameters: {'max_depth': 7, 'eta': 0.2931054837989315, 'subsample': 0.6464602520115516, 'colsample_bytree': 0.6531490027396228}.


Pareto-optimal solutions:
Params: {'alpha': 0, 'max_depth': 4, 'eta': 0.23427942936231477, 'subsample': 0.6945965931961444, 'colsample_bytree': 0.6527247741457783}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.06, 'max_depth': 6, 'eta': 0.2462887362148314, 'subsample': 0.7148511211224349, 'colsample_bytree': 0.8641681121879333}, Accuracy: 0.6607, DPD: 0.0163
Params: {'alpha': 0.06, 'max_depth': 4, 'eta': 0.2047769975940788, 'subsample': 0.6620114429622679, 'colsample_bytree': 0.9637821329925713}, Accuracy: 0.6629, DPD: 0.0185
Params: {'alpha': 0.06, 'max_depth': 3, 'eta': 0.22811636933151896, 'subsample': 0.6573898483717654, 'colsample_bytree': 0.9298562002671199}, Accuracy: 0.6622, DPD: 0.0175
Params: {'alpha': 0.06, 'max_depth': 3, 'eta': 0.2235453583133459, 'subsample': 0.658487905755447, 'colsample_bytree': 0.9372928122625637}, Accuracy: 0.6614, DPD: 0.0165
Params: {'alpha': 0.06, 'max_depth': 3, 'eta': 0.1555700702666115, 'subsample': 0.9851385794665797, 'colsample_bytree': 0

In [13]:
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()